In [26]:
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
import sys
import os.path

# Устанавливаем seed для повторяемости результатов
np.random.seed(42)

TRAIN_FILE = "/home/kulikov/Downloads/train (1).csv"
TEST_FILE = "/home/kulikov/Downloads/test (2).csv"
OUTPUT_FILE = "submission.csv"

# Размер изображения
img_rows, img_cols = 28, 28

if not os.path.isfile(TRAIN_FILE) or not os.path.isfile(TEST_FILE):
    print("""Загрузите с kaggle данные для обучения и тестирования
             (файлы train.csv и test.csv) и запишите их в текущий каталог
          """)
    sys.exit()


# Загружаем данные для обучения
train_dataset = np.loadtxt(TRAIN_FILE, skiprows=1, dtype='int', delimiter=",")
# Выделяем данные для обучения
x_train = train_dataset[:, 1:]
# Переформатируем данные в 2D, бэкенд Theano
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)
# Нормализуем данные
x_train = x_train.astype("float32")
x_train /= 255.0
# Выделяем правильные ответы
y_train = train_dataset[:, 0]
# Преобразуем правильные ответы в категоризированное представление
y_train = np_utils.to_categorical(y_train)



#Загружаем данные для предсказания
test_dataset = np.loadtxt(TEST_FILE, skiprows=1, delimiter=",")
# Переформатируем данные в 2D, бэкенд Theano
x_test = test_dataset.reshape(test_dataset.shape[0], img_rows, img_cols, 1)
x_test /= 255.0

In [28]:
x_test.shape

(28000, 28, 28, 1)

In [29]:
# Создаем последовательную модель
model = Sequential()

model.add(Conv2D(75, 5, 5,
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(100, 5, 5, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))


# Компилируем модель
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

print(model.summary())

/home/kulikov/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(75, (5, 5), activation="relu", input_shape=(28, 28, 1...)`
  
/home/kulikov/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(100, (5, 5), activation="relu")`
  if __name__ == '__main__':


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 24, 24, 75)        1950      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 75)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 75)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 8, 8, 100)         187600    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 4, 100)         0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 4, 4, 100)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)              0         
__________

In [30]:
# Обучаем сеть
model.fit(x_train, y_train, batch_size=200, epochs=10, verbose=2)

Epoch 1/10
275s - loss: 0.2766 - acc: 0.9129
Epoch 2/10
248s - loss: 0.0716 - acc: 0.9783
Epoch 3/10
276s - loss: 0.0501 - acc: 0.9847
Epoch 4/10
234s - loss: 0.0396 - acc: 0.9873
Epoch 5/10
275s - loss: 0.0342 - acc: 0.9889
Epoch 6/10
247s - loss: 0.0280 - acc: 0.9912
Epoch 7/10
287s - loss: 0.0239 - acc: 0.9922
Epoch 8/10
285s - loss: 0.0197 - acc: 0.9935
Epoch 9/10
212s - loss: 0.0210 - acc: 0.9930
Epoch 10/10
207s - loss: 0.0184 - acc: 0.9938


In [31]:
# Making predictions
predictions = model.predict(x_test)
# Converting from categorical to classed
predictions = np.argmax(predictions, axis=1)

# Saving predictions to the file
out = np.column_stack((range(1, predictions.shape[0]+1), predictions))
np.savetxt(OUTPUT_FILE, out, header="ImageId,Label", comments="", fmt="%d,%d")

In [32]:
model.save_weights("conv_w.hdf")

In [60]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers, models
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense

model2 = applications.VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))
print('Model loaded.')

Model loaded.


In [50]:
model2.output_shape[1:]

(1, 1, 512)

In [55]:
top_model = Sequential()
top_model.add(Flatten(input_shape=model2.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(10, activation='softmax'))



In [58]:
model2.add(top_model)

AttributeError: 'Model' object has no attribute 'add'

In [61]:
model3 = models.Model(inputs=model2.input, outputs=top_model(model2.output))

In [62]:
for layer in model3.layers[:25]:
    layer.trainable = False

In [63]:
model3.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [66]:
import cv2

ModuleNotFoundError: No module named 'cv2'

In [65]:
x_train2 = [cv2.cvtColor(cv2.resize(i, (48,48)), cv2.COLOR_GRAY2BGR) for i in x_train]
x_train2 = np.concatenate([arr[np.newaxis] for arr in x_train2]).astype('float32')

x_test2 = [cv2.cvtColor(cv2.resize(i, (48,48)), cv2.COLOR_GRAY2BGR) for i in x_test]
x_test2 = np.concatenate([arr[np.newaxis] for arr in x_test2]).astype('float32')

NameError: name 'cv2' is not defined

In [47]:
model3.fit(x_train, y_train, batch_size=200, epochs=10, verbose=2)




ValueError: Error when checking input: expected input_2 to have shape (None, 48, 48, 3) but got array with shape (42000, 28, 28, 1)